In [1]:
import json

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_MATCH = """
## Instruction

You are an expert in analyzing user preferences based on their movie interactions. The following records include conversations and clicks, and they **belong to the same user**. Your task is to **think step by step** to infer this user's movie preferences.  
If the records do not provide enough information to infer preferences, respond with: "I don't know."

### **Important:**  
There are two parts to your output:
1. **Analysis**:  
   - Provide a step-by-step breakdown of how the user's interactions reveal their preferences.  
   - Follow the order of interaction records and keep the analysis **brief and clear**.
2. **User Preferences**:  
   - Summarize the user's preferences in a **structured format** with two main categories: `movies` and `attributes`.  
   - `movies` refers to specific movies or series the user likes or dislikes.  
   - `attributes` refers to general movie characteristics such as genres, themes, styles, directors, writers, stars, plots, etc.
   - If there is **insufficient information** to infer preferences for a category, output **None** for that category (e.g., `movies: like: None` or `attributes: like: None`).

### Input Format:
User interaction records are structured as follows:
[interactionId]. 
[content]

## Output Format:
### Analysis:
[Follow the interaction records **in order** and provide a **brief and clear** step-by-step analysis. The analysis shouldn't be too long.]

After completing the step-by-step analysis, provide a final preference summary using **only the following format**:
### User Preferences:
- movies:
  - like: movie or series A (year), movie or series B (year), ..., or None
  - dislike: movie or series C (year), ..., or None
- attributes:
  - like: preferred attributes. A, preferred attributes. B, ..., or None if not enough information
  - dislike: disliked attributes. C, ..., or None if not enough information

**Important rules**:
- Each movie should be in the format: title (year), separated by ",".
- Do not use quotation marks (" "), square brackets ([ ]), or markdown symbols (*) to identify movies and attributes when listing movies and attributes.
- You should respond in English.
""".strip()

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_UPDATE_FORMAT = """
## Instruction

You are an expert in analyzing user preferences based on their movie interactions. The provided interaction records **belong to the same user**. Your task is to **think step by step** to infer this user's movie preferences.  
If the records do not provide enough information to infer preferences, respond with: "I don't know."

### **Important:**  
There are two parts to your output:
1. **Analysis**:  
   - Provide a step-by-step breakdown of how the user's interactions reveal their preferences.  
   - Follow the order of interaction records and keep the analysis **brief and clear**.
2. **User Preferences**:  
   - Summarize the user's preferences in a **structured format** with two main categories: `movies` and `attributes`.  
   - `movies` refers to specific movies or series the user likes or dislikes.  
   - `attributes` refers to general movie characteristics such as genres, themes, styles, directors, writers, stars, plots, etc.
   - If there is **insufficient information** to infer preferences for a category, output **None** for that category (e.g., `movies: like: None` or `attributes: like: None`).

### Input Format:
User interaction records are structured as follows:
[interactionId]. [action]  
[content]

## Output Format:
### Analysis:
[Follow the interaction records **in order** and provide a **brief and clear** step-by-step analysis. The analysis shouldn't be too long.]

After completing the step-by-step analysis, provide a final preference summary using **only the following format**:
### User Preferences:
- movies:
  - like: list of movies or series the user enjoys, or None if not enough information
  - dislike: list of movies or series the user dislikes, or None if not enough information
- attributes:
  - like: list of preferred attributes such as genres, themes, styles, directors, etc., or None if not enough information
  - dislike: list of disliked attributes, or None if not enough information

## Attention:
You should respond in English.
""".strip()

In [ ]:
RECOMMEND_INSTRUCTION_QWEN_NEW_MATCH = """
## Instruction

You are an expert in analyzing user preferences based on their movie interactions. The following records include conversations and clicks, and they **belong to the same user**. Your task is to **think step by step** to infer this user's movie preferences.  
If the records do not provide enough information to infer preferences, respond with: "I don't know."

### **Important:**  
There are two parts to your output:
1. **Analysis**:  
   - Provide a step-by-step breakdown of how the user's interactions reveal their preferences.  
   - Follow the order of interaction records and keep the analysis **brief and clear**.
2. **User Preferences**:  
   - Summarize the user's preferences in a **structured format** with two main categories: `movies` and `attributes`.  
   - `movies` refers to specific movies or series the user likes or dislikes.  
   - `attributes` refers to general movie characteristics such as genres, themes, styles, directors, writers, stars, plots, etc.
   - If there is **insufficient information** to infer preferences for a category, output **None** for that category (e.g., `movies: like: None` or `attributes: like: None`).

### Input Format:
User interaction records are structured as follows:
[Interaction Id]. 
[content]

## Output Format:
### Analysis:
[Follow the interaction records **in order** and provide a **brief and clear** step-by-step analysis. The analysis shouldn't be too long.]

After completing the step-by-step analysis, provide a final preference summary using **only the following format**:
### User Preferences:
- movies:
  - like: movie or series A (year), movie or series B (year), ..., or None
  - dislike: movie or series C (year), ..., or None
- attributes:
  - like: preferred attributes. A, preferred attributes. B, ..., or None if not enough information
  - dislike: disliked attributes. C, ..., or None if not enough information

**Important rules**:
- Each movie should be in the format: title (year), separated by ",".
- Do not use quotation marks (" "), square brackets ([ ]), or markdown symbols (*) to identify movies and attributes when listing movies and attributes.
- You should respond in English.
""".strip()